# Pénzügyi adatelemzés 
### Második házi feladat
### Garbage in - garbage out

A házi feladatot itt a notebook-ban kell megoldani
- legfeljebb 4 fős csapatokban és
- ipynb (Jupyter Notebook) formátumban a Moodle-re feltölteni <span style="color:red">**2025-11-09 vasárnap este 23:55-ig**</span> csapatonként EGY példányban (tehát NE töltse fel mindenki).
- A csapatokra vonatkozólag nincs más megkötés, mint, hogy legfeljebb 4 fősek lehetnek, lehet dolgozni akár párban, akár egyedül is, nem szükséges a csapattagoknak ugyanabba a szemináriumi csoportba járni, és eltérhetnek az első házi feladat csapataitól. 
- A csapattagok nevét, Neptun-kódját és "uni-corvinus"-os e-mail címét írjátok be a notebook második cellájába. Minden feltöltött file-ra az a hallgató fog pontot kapni, akinek az adatai itt hibátlanul szerepelnek. Ha ugyanaz a csapat több file-t is feltölt, akkor pontot vonunk le. 

Minden kérdés után van a szöveges válasznak vagy kódnak egy üres cella, de természetesen lehet újabb cellákat is beszúrni és további függvényeket írni.

Nem feltétlenül elegendő az előre megírt függvényeket kiegészíteni, a kódnak minden, a feladatben kért output-ot el kell készítenie és a cellákra sorban kattintva hibátlanul lefutnia. A feladat értékelésénél nem fogunk semmilyen további kódot hozzáírni, ha valaki elfelejt meghívni egy függvényt, akkor pontot veszíthet. Mivel a notebook egyetlen session, emlékszik minden változóra, akkor is, ha az azt létrehozó kódot később kitöröljük, így ellenőrzésként célszerű a végső verzió tesztelése előtt újraindítani a kernel-t vagy bezárni a file-t újraindítani a szervert, és úgy kipróbálni.

Az elméleti kérdésekre adott válaszok legyenek rövidek, pontosak, lényegre törőek, a szükséges mennyiségű, de nem több indoklással. Aki válaszként mindent is leír, (ahogy a chatGPT szokta), azt negatívan értékeljük. Az ábrákat nem kell "szép"-re formázni, megfelelnek az alapértelmezett beállítások. A kódok a kért feladatokat oldják meg, ne valami mást vagy általánosabbat, a számítások egyes lépéseit szükség esetén egy-két szavas kommentekkel el lehet látni.

AI indokolt esetben és módon használható, de meg kell jelölni, hogy hol, mire és miért használtátok. Ha Python kód generálásra használtok generatív AI-t, akkor másoljátok ki a promt-ot (az AI-nak adott utasítást) egy külön cellába.

A szimulációk paramétereit a beadott file-ban úgy állítsátok be, hogy a kódok futása gyors (legfeljebb néhány másodperc) legyen, de a szöveges válaszokban összefoglalhatjátok a szimulációk eredményeit nagyobb mintaelemszámra is. Sokáig futó kódhoz célszerű a `tqdm` package `tqdm` függvényét használni, ami egy szépen formázott progressbart készít egy `for` ciklushoz. 

**Csapattagok (név, Neptun-kód, e-mail cím):**

A feladatok célja azt illusztrálni, hogyan hat a regressziós modellre ha "ész nélkül" minden változót beleteszünk, ami rendelkezésre áll.  


In [1]:
# A cellákban külön importokat ne tegyünk, egy helyen töltsük be az összes felhasznált package-t. 
# Az összes feladat megoldható ezekkel (de persze lehet bővíteni a listát):

from matplotlib import pyplot as plt
import numpy as np
import statsmodels.api as sm
import seaborn as sns
from tqdm import tqdm

#### 1. Feladat

Legyen $k$, $l$ és $m$ pozitív egész számok, $x_1, x_2, \dots, x_k$ és $\varepsilon$ független standard normális eloszlású valószínűségi változók, $\sigma\in\mathbb{R}$ egy pozitív szám és tekintsük az
\begin{equation}
Y = \sum_{i = 1}^k \frac{(k-i+1)}{k} x_i + \sigma\varepsilon
\tag{E}
\end{equation}
regressziós egyenletet. Az adatgeneráló folyamat tehát tartalmaz $k$ darab magyarázó változót egyre csökkenő bétákkal tehát, például $k=5$-re $$Y = x_1 + \frac{4}{5}x_2 + \frac{3}{5}x_3 + \frac{2}{5}x_4 + \frac{1}{5}x_5 + \sigma\varepsilon$$.

Írjunk egy függvényt, ami adott $m$, $k$, $l$ és $\sigma$ függvényében generál egy olyan adattáblát, amely $m$ darab megfigyelést ($m$ sort) tartalmaz 

- a fenti $k$ darab független standard normális eloszlású magyarázó változóból,
- $l$ darab ún. zajváltozóból (melyek mind egymástól, mind a többi $x_i$-től független standard normális eloszlású változók),
- és az $(E)$ egyenlet szerinti célváltozóból, melyeknek tehát semmi közük a zajváltozókhoz.

Az adattáblát a későbbi kényelmesebb használat érdekében úgy is létrehozhatjuk, hogy csak a magyarázó változókat és a zajváltozókat (tehát csak $x$-eket) tartalmazza, a célváltozót pedig külön egy `tuple` második elemeként adja vissza a függvény.  


In [130]:
def generate_data(m, k, l, sigma=1):
    x=np.random.normal(size=(m,k))
    z=np.random.normal(size=(m,l))
    e=np.random.normal(size=m)
    beta=np.array([(k-i+1)/k for i in range(1, k+1)])
    y=x@beta+sigma*e
    X=np.hstack((x,z))
    return X, y

# print(generate_data(100, 2, 2))

#### 2. Feladat

Írjunk egy függvényt, amely az adatok és egy $\alpha$ signifikanciaszint alapján megbecsüli a regressziós modellt és visszaad egy $k+l$ elemű vektort (listát vagy tuple-t vagy numpy array-t), amely az összes magyarázó változó (a zajváltozókat is beleértve) $t$-tesztjének $p$-értékei alapján egy logikai értékekből álló vektort ad vissza, annak megfelelően, hogy az adott változó $t$-tesztje a megadott $\alpha$ szinten szignifikáns-e.

Elméleti kérdés: egy "tökéletes világban" milyen eredményeket (milyen True/False értékeket) kéne ennek a függvénynek visszaadni?

<div style="color: blue">
Válasz: Egy "tökéletes világban", ahol a minta elemszáma nagy (esetleg végtelen), a magyarázó változókra $x_i$-kre minden esetben True értéket kellene kapnunk (azaz szignifikánsak), és a zajváltozókat a nagy minta elemszám miatt ki tudjuk szűrni, ezért azokra False (nem szignifikáns) értéket várunk.
</div>

In [133]:
def beta_sign_test(data, alpha=0.95):
    X, y = data
    X = sm.add_constant(X) # konstanssal való regresszálásnál
    model = sm.OLS(y, X).fit()

    pstat = model.pvalues

    truefalse = []
    # print(pstat)
    
    for p in pstat:
        if p < (1-alpha):
            truefalse.append(True)
        else:
            truefalse.append(False)

    truefalse.pop(0) # konstans törlése
    return truefalse

# print(beta_sign_test(generate_data(100, 3, 3)))

#### 3. Feladat

Legyen az egyszerűség kedvéért $\alpha = 95\%$, $\sigma = 1$ és $k = 5$. Futtassunk szimulációt a 2. feladatban megírt függvény használatával az $l$ (a zajváltozók száma) és az $m$ (megfigyelések száma) különböző kombinációira, és próbáljuk meg szimulációval megállapítani, hogy adott számú zajváltozó esetén hány megfigyelés kell ahhoz, hogy "nagy valószínűséggel jó eredményt" kapjunk a feladatból, tehát mondjuk 90%-os valószínűséggel a modell válassza szét, a zajváltozókat és a releváns változókat. 

Megjegyzés: Itt nem várunk feltétlenül tökéletesen egzakt megoldást, de mondjatok valami értelmeset arról, hogy mennyi adat biztos, hogy nem elég a megbízható eredményhez, és mennyi adat lesz már nagyon nagy valószínűséggel elég. A főbb megállapításokat foglaljátok össze szövegesen, illetve további opcionális (nem kötelező) feladatként készítsetek három  táblázatot, amelyben különböző $l$ és $m$ kombinációkra szimuláció alapján annak a valószínűsége van, hogy

- a modell legalább 1 releváns magyarázó változót megtalál,
- a modell nem talál szignifikánsnak egyetlen zajváltozót sem,
- a modell felismeri az adatgeneráló folyamatot, azaz szét tudja választani a releváns magyarázó változókat a zajtól.

Azt itt nem kell értékelni, hogy mekkora hibával találja meg a modell bétákat, csak azt, hogy melyiket tekinti nullának illetve nem nullának a $t$-teszt). A táblázatot olyan $l$ és $m$ kombinációkból rakjátok össze, hogy abban a releváns eredmények látszódjanak, tehát nemtriviális átmenetek nem csak 0-k és 1-esek.

In [134]:
k = 5
sim_size = 500

def sim_szetvalaszt(m, k, l, alpha=0.95):
    exp = []
    for i in range(k):
        exp.append(True)
    for i in range(l):
        exp.append(False)
    
    res = beta_sign_test(generate_data(m, k, l), alpha)
    if res == exp:
        return True
    else:
        return False
        
def sim(m, k, l, sim_size, alpha=0.95):
    talalt = [sim_szetvalaszt(m, k, l) for _ in range(sim_size)]
    szazalek = sum(talalt)/len(talalt)
    return szazalek

for m in range(1,11):
    print(f"{m*100} db elemő minta esetén a elvárt kiszűrések százaléka ({sim_size} nagyságú szimulációra):")
    for l in range(0,4):
        print(f"{l} db zajváltozóra: {round(sim(m*100,k,l,sim_size)*100, 4)}%")
    print("")

print("(Az elvárt kiszűrés az, ahol a magyarázó változók paraméterei szignifikánsak, és a zajváltozók paraméterei közül egyik sem szignifikáns)")

100 db elemő minta esetén a elvárt kiszűrések százaléka (500 nagyságú szimulációra):
0 db zajváltozóra: 43.6%
1 db zajváltozóra: 48.2%
2 db zajváltozóra: 44.4%
3 db zajváltozóra: 37.6%

200 db elemő minta esetén a elvárt kiszűrések százaléka (500 nagyságú szimulációra):
0 db zajváltozóra: 79.2%
1 db zajváltozóra: 75.0%
2 db zajváltozóra: 70.2%
3 db zajváltozóra: 68.0%

300 db elemő minta esetén a elvárt kiszűrések százaléka (500 nagyságú szimulációra):
0 db zajváltozóra: 92.8%
1 db zajváltozóra: 88.2%
2 db zajváltozóra: 84.4%
3 db zajváltozóra: 79.0%

400 db elemő minta esetén a elvárt kiszűrések százaléka (500 nagyságú szimulációra):
0 db zajváltozóra: 97.2%
1 db zajváltozóra: 92.0%
2 db zajváltozóra: 89.4%
3 db zajváltozóra: 83.0%

500 db elemő minta esetén a elvárt kiszűrések százaléka (500 nagyságú szimulációra):
0 db zajváltozóra: 99.8%
1 db zajváltozóra: 96.0%
2 db zajváltozóra: 90.2%
3 db zajváltozóra: 84.0%

600 db elemő minta esetén a elvárt kiszűrések százaléka (500 nagyságú 

<div style="color:blue">
Konklúziók

A szimulációnk alapján azt tapasztaltuk, hogy az 5 magyarázó változót és 1 zajváltozót tartalmazó modell kb. 350/400 darab körüli minta esetén már 90% valószínűséggel szét tudja választani a magyarázó- és zajváltozókat. Viszont ha bevonunk meg egy újabb zajváltozót, akkor ehhez a 90%-os szétválasztási sikerességhez a szimulációnk szerint több mint 800 elemű mintára lenne szükésgünk. A harmadik zajváltozó bevonásánál már több mint 10000 elemű mintára lenne szükség.

A szimuláció megerősítette a második feladatban lévő kérdésre adott válaszunkat, mivel a minta elemszám növekedésével valóban nagyobb valószínűséggel tudjuk kiszűrni a magyarázó változókat a zajváltozóktól.

A $k$ növelésével ez még extrémebb lenne, mivel a $k$-adik változó $\beta$ értéke már az adatgeneráló folyamat közben tart a 0-ba, kisebb mértékben befolyásolja a generált $Y$-okat, ezért nagyobb valószínűséggel tekinti a t-test az új valós magyarázó változót is zajváltozónak.
</div>